In [ ]:
! pip install streamlit

In [ ]:
! pip install pyngrok

In [3]:
import tensorflow as tf
import keras
import numpy as np

In [47]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import keras
import numpy as np
from PIL import Image, ImageOps

st.set_option('deprecation.showfileUploaderEncoding', False)
@st.cache(allow_output_mutation=True)
def load_model():
    mobile_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/MobileNet')
    basic_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/Basic CNN/ver0_additional')
    alexNet_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/AlexNet')
    vgg_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/VggNet')
    googLenet_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/GoogLeNet')
    models = [mobile_model, basic_model, alexNet_model, vgg_model, googLenet_model]
    return models

def image_prepare(image_data):

    image150 = ImageOps.fit(image_data, (150,150), Image.ANTIALIAS)
    image224 = ImageOps.fit(image_data, (224,224), Image.ANTIALIAS)
    image227 = ImageOps.fit(image_data, (227,227), Image.ANTIALIAS)

    img_arr150 = keras.preprocessing.image.img_to_array(
        img = image150
    )

    img_arr224 = keras.preprocessing.image.img_to_array(
        img = image224
    )

    img_arr227 = keras.preprocessing.image.img_to_array(
        img = image227
    )

    img_arr150 = np.array([img_arr150]) / 255.
    img_arr224 = np.array([img_arr224]) / 255.
    img_arr227 = np.array([img_arr227]) / 255.
    
    return img_arr150 , img_arr224, img_arr227

def import_and_predict(image_data, models):
    img_arr150 , img_arr224, img_arr227 = image_prepare(image_data)

    mob = models[0].predict(img_arr224)
    bas = models[1].predict(img_arr150)
    alx = models[2].predict(img_arr227)
    vgg = models[3].predict(img_arr224)
    goo = models[4].predict(img_arr224)
    
    pred = (mob + bas + alx + vgg + goo[0]) / 5
    preds = [mob, bas, alx, vgg , goo]
    return pred , preds

models = load_model()

st.write("""# 고양이 vs 개 분류기""")

file_ = st.file_uploader('이미지를 업로드 해주세요!', type=['jpg', 'png'])

if file_ is None:
    st.text('이미지 파일을 업로드 해주세요!')
else :
    image = Image.open(file_)
    st.image(image, use_column_width = True)
    pred, preds = import_and_predict(image, models)
    pred = np.array(pred[0])
    pred = tf.argmax(pred)
    labels = ['고양이' , '개']
    string = f'알파고님들은 합의 끝에 이 사진을 {labels[pred]} 라고 판단했습니다.'
    st.success(string)
    left_column, right_column = st.beta_columns(2)
    pressed = left_column.button(f'추정 확률 공개')
    if pressed:
        st.write(f'MobileNet님 : {round(preds[0][0][pred] * 100 , 4)}% 로 이것은 {labels[pred]}임.')
        st.write(f'Trash님 : {round(preds[1][0][pred] * 100, 4)}% 로 이것은 {labels[pred]}임.')
        st.write(f'AlexNet님 : {round(preds[2][0][pred] * 100, 4)}% 로 이것은 {labels[pred]}임.')
        st.write(f'Vgg16님 : {round(preds[3][0][pred] * 100, 4)}% 로 이것은 {labels[pred]}임.')
        st.write(f'GoogLeNet님 : {round(preds[4][0][0][pred] * 100, 4)}% 로 이것은 {labels[pred]}임.')

Overwriting app.py


In [ ]:
!ls

In [23]:
!ngrok authtoken *

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [24]:
from pyngrok import ngrok

In [25]:
!streamlit run app.py&>/dev/null&

In [26]:
publ_url = ngrok.connect(addr='8501')

In [27]:
publ_url

<NgrokTunnel: "http://898f39b29665.ngrok.io" -> "http://localhost:8501">

In [ ]:
!ps

In [49]:
!kill 657

In [50]:
ngrok.kill()